In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

from enum import Enum

In [2]:
#存檔位置
ModelMode = "ResNet_ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[4] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[4] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[4]

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df[df.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#殘差網路模塊
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        shortcut = self.shortcut(input)
        input = nn.ReLU()(self.bn1(self.conv1(input)))
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = input + shortcut
        return nn.ReLU()(input)
    
#ResNet_18+注意力機制 模型
class ResNet_18(nn.Module):
    def __init__(self, nums_class=10):
        super(ResNet_18, self).__init__()
        
        self.layer_0 = nn.Sequential(
            # width_of_output = (width_of_input - filter_size + 2*padding)/stride + 1
            nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = (7,7), stride = (2,2), padding = 3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        #殘差層
        #-----------------------------------------------------
        self.layer_1 = nn.Sequential(
            ResBlock(64, 128, downsample=True),
            ResBlock(128, 128, downsample=False)
        )
        
        self.layer_2 = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.Linear(64, nums_class)
        )
        
    def forward(self, x):
        x = self.layer_0(x)
        x = self.layer_1(x)
        x = self.layer_2(x)
        return x



In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 60
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = 0.0032
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_13316\2625783892.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 132.61 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = ResNet_18(nums_class = 50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #5s
            dataX = dataX5
            dataY = dataY5
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
        else:
            #1s
            dataX = dataX1
            dataY = dataY1
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 2.8832478523254395 Train_acc: 10.375000%
Train Epoch: 2/60 Train_Loss: 2.641758918762207 Train_acc: 21.937500%
Train Epoch: 3/60 Train_Loss: 2.3640847206115723 Train_acc: 26.625000%
Train Epoch: 4/60 Train_Loss: 2.098355770111084 Train_acc: 28.312500%
Train Epoch: 5/60 Train_Loss: 2.0893518924713135 Train_acc: 34.437500%
Train Epoch: 6/60 Train_Loss: 1.918717384338379 Train_acc: 37.312500%
Train Epoch: 7/60 Train_Loss: 2.225602149963379 Train_acc: 40.750000%
Train Epoch: 8/60 Train_Loss: 2.0583982467651367 Train_acc: 43.500000%
Train Epoch: 9/60 Train_Loss: 1.7640066146850586 Train_acc: 46.250000%
Train Epoch: 10/60 Train_Loss: 1.7959386110305786 Train_acc: 50.750000%
Train Epoch: 11/60 Train_Loss: 1.8958

Train Epoch: 46/60 Train_Loss: 0.18277959525585175 Train_acc: 96.770836%
Train Epoch: 47/60 Train_Loss: 0.06285424530506134 Train_acc: 97.166672%
Train Epoch: 48/60 Train_Loss: 0.08677909523248672 Train_acc: 97.041672%
Train Epoch: 49/60 Train_Loss: 0.27318161725997925 Train_acc: 96.979172%
Train Epoch: 50/60 Train_Loss: 0.06430856883525848 Train_acc: 97.104172%
Train Epoch: 51/60 Train_Loss: 0.20472352206707 Train_acc: 97.187500%
Train Epoch: 52/60 Train_Loss: 0.09310205280780792 Train_acc: 97.125000%
Train Epoch: 53/60 Train_Loss: 0.36690041422843933 Train_acc: 97.250000%
Train Epoch: 54/60 Train_Loss: 0.06558933854103088 Train_acc: 97.166672%
Train Epoch: 55/60 Train_Loss: 0.010851876810193062 Train_acc: 97.166672%
Train Epoch: 56/60 Train_Loss: 0.07450918108224869 Train_acc: 97.166672%
Train Epoch: 57/60 Train_Loss: 0.11545006930828094 Train_acc: 97.250000%
Train Epoch: 58/60 Train_Loss: 0.029582731425762177 Train_acc: 97.125000%
Train Epoch: 59/60 Train_Loss: 0.0778936967253685 Tr

Train Epoch: 29/60 Train_Loss: 0.41249531507492065 Train_acc: 81.750000%
Train Epoch: 30/60 Train_Loss: 0.5990321636199951 Train_acc: 83.125000%
Train Epoch: 31/60 Train_Loss: 0.4573134183883667 Train_acc: 85.500000%
Train Epoch: 32/60 Train_Loss: 0.639518141746521 Train_acc: 86.187500%
Train Epoch: 33/60 Train_Loss: 0.30498290061950684 Train_acc: 88.625000%
Train Epoch: 34/60 Train_Loss: 0.4311663806438446 Train_acc: 90.125000%
Train Epoch: 35/60 Train_Loss: 0.4159746766090393 Train_acc: 90.187500%
Train Epoch: 36/60 Train_Loss: 0.29575783014297485 Train_acc: 90.500000%
Train Epoch: 37/60 Train_Loss: 0.35022151470184326 Train_acc: 90.875000%
Train Epoch: 38/60 Train_Loss: 0.3376285135746002 Train_acc: 91.375000%
Train Epoch: 39/60 Train_Loss: 0.3813679814338684 Train_acc: 91.375000%
Train Epoch: 40/60 Train_Loss: 0.3053014278411865 Train_acc: 91.062500%
Train Epoch: 41/60 Train_Loss: 0.3353756368160248 Train_acc: 91.562500%
Train Epoch: 42/60 Train_Loss: 0.3478231430053711 Train_acc: 

Train Epoch: 12/60 Train_Loss: 0.6872735619544983 Train_acc: 84.062508%
Train Epoch: 13/60 Train_Loss: 0.8640305399894714 Train_acc: 85.675003%
Train Epoch: 14/60 Train_Loss: 0.5776025652885437 Train_acc: 85.225006%
Train Epoch: 15/60 Train_Loss: 0.43617144227027893 Train_acc: 86.312508%
Train Epoch: 16/60 Train_Loss: 0.13274814188480377 Train_acc: 89.825005%
Train Epoch: 17/60 Train_Loss: 0.4256870448589325 Train_acc: 91.062508%
Train Epoch: 18/60 Train_Loss: 0.5640199780464172 Train_acc: 91.475006%
Train Epoch: 19/60 Train_Loss: 0.35980477929115295 Train_acc: 91.425003%
Train Epoch: 20/60 Train_Loss: 0.4488886892795563 Train_acc: 91.525002%
Train Epoch: 21/60 Train_Loss: 0.5829458832740784 Train_acc: 91.700005%
Train Epoch: 22/60 Train_Loss: 0.31915199756622314 Train_acc: 91.737503%
Train Epoch: 23/60 Train_Loss: 0.1953606903553009 Train_acc: 91.087502%
Train Epoch: 24/60 Train_Loss: 0.32461538910865784 Train_acc: 90.862503%
Train Epoch: 25/60 Train_Loss: 0.1283683329820633 Train_acc

Train Epoch: 60/60 Train_Loss: 0.07467397302389145 Train_acc: 97.562500%
Test accuracy of the model: 39.450000%
Test accuracy of the model: 60.166667%
Test accuracy of the model: 70.000000%
Training Time: 43.34 seconds
----------------------------------------------------------------------------------------------------------------
fold:  3 TL:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 0.9997434616088867 Train_acc: 66.604172%
Train Epoch: 2/60 Train_Loss: 0.882463812828064 Train_acc: 75.020836%
Train Epoch: 3/60 Train_Loss: 0.7724865078926086 Train_acc: 75.812500%
Train Epoch: 4/60 Train_Loss: 0.6398894190788269 Train_acc: 79.583336%
Train Epoch: 5/60 Train_Loss: 0.7358832955360413 Train_acc: 79.354172%
Train Epoch: 6/60 Train_Loss: 0.5443295240402222 Train_acc: 80.312500%
Train Epoch: 7/60 Train_Loss: 0.6563966870307922 Train_acc: 82.395836%
Train Epoch: 8/60 Train_Loss: 0.82533907890

Train Epoch: 43/60 Train_Loss: 0.24516934156417847 Train_acc: 92.787506%
Train Epoch: 44/60 Train_Loss: 0.21467071771621704 Train_acc: 92.875008%
Train Epoch: 45/60 Train_Loss: 0.15250395238399506 Train_acc: 93.050003%
Train Epoch: 46/60 Train_Loss: 0.26446184515953064 Train_acc: 92.837502%
Train Epoch: 47/60 Train_Loss: 0.3434515595436096 Train_acc: 92.912506%
Train Epoch: 48/60 Train_Loss: 0.30811211466789246 Train_acc: 93.000008%
Train Epoch: 49/60 Train_Loss: 0.2606143653392792 Train_acc: 93.037506%
Train Epoch: 50/60 Train_Loss: 0.27539321780204773 Train_acc: 92.887505%
Train Epoch: 51/60 Train_Loss: 0.22920869290828705 Train_acc: 92.825005%
Train Epoch: 52/60 Train_Loss: 0.05602812394499779 Train_acc: 92.925003%
Train Epoch: 53/60 Train_Loss: 0.2048797905445099 Train_acc: 93.037506%
Train Epoch: 54/60 Train_Loss: 0.28080156445503235 Train_acc: 92.987503%
Train Epoch: 55/60 Train_Loss: 0.1894984096288681 Train_acc: 93.050003%
Train Epoch: 56/60 Train_Loss: 0.38186052441596985 Trai

Train Epoch: 26/60 Train_Loss: 0.6493582129478455 Train_acc: 92.750000%
Train Epoch: 27/60 Train_Loss: 0.16829338669776917 Train_acc: 93.583336%
Train Epoch: 28/60 Train_Loss: 0.2854175269603729 Train_acc: 93.604172%
Train Epoch: 29/60 Train_Loss: 0.3828887343406677 Train_acc: 94.833336%
Train Epoch: 30/60 Train_Loss: 0.21760907769203186 Train_acc: 94.791672%
Train Epoch: 31/60 Train_Loss: 0.1449134349822998 Train_acc: 95.895836%
Train Epoch: 32/60 Train_Loss: 0.23488351702690125 Train_acc: 96.041672%
Train Epoch: 33/60 Train_Loss: 0.1834641546010971 Train_acc: 96.354172%
Train Epoch: 34/60 Train_Loss: 0.2506013512611389 Train_acc: 95.875000%
Train Epoch: 35/60 Train_Loss: 0.13272155821323395 Train_acc: 96.125000%
Train Epoch: 36/60 Train_Loss: 0.33599305152893066 Train_acc: 96.250000%
Train Epoch: 37/60 Train_Loss: 0.192339688539505 Train_acc: 96.291672%
Train Epoch: 38/60 Train_Loss: 0.22269289195537567 Train_acc: 96.229172%
Train Epoch: 39/60 Train_Loss: 0.036600902676582336 Train_a

Train Epoch: 9/60 Train_Loss: 1.8902119398117065 Train_acc: 42.937500%
Train Epoch: 10/60 Train_Loss: 1.7269036769866943 Train_acc: 44.562500%
Train Epoch: 11/60 Train_Loss: 1.668364405632019 Train_acc: 50.375000%
Train Epoch: 12/60 Train_Loss: 1.8058784008026123 Train_acc: 51.625000%
Train Epoch: 13/60 Train_Loss: 1.5852739810943604 Train_acc: 52.750000%
Train Epoch: 14/60 Train_Loss: 1.2736468315124512 Train_acc: 58.937500%
Train Epoch: 15/60 Train_Loss: 1.3308693170547485 Train_acc: 58.812500%
Train Epoch: 16/60 Train_Loss: 0.9638014435768127 Train_acc: 63.750000%
Train Epoch: 17/60 Train_Loss: 1.2081432342529297 Train_acc: 68.187500%
Train Epoch: 18/60 Train_Loss: 1.0631612539291382 Train_acc: 70.687500%
Train Epoch: 19/60 Train_Loss: 1.0453171730041504 Train_acc: 68.687500%
Train Epoch: 20/60 Train_Loss: 0.9606730937957764 Train_acc: 71.062500%
Train Epoch: 21/60 Train_Loss: 0.8038408160209656 Train_acc: 72.812500%
Train Epoch: 22/60 Train_Loss: 0.9237032532691956 Train_acc: 73.12

Train Epoch: 57/60 Train_Loss: 0.10226128995418549 Train_acc: 96.979172%
Train Epoch: 58/60 Train_Loss: 0.049558598548173904 Train_acc: 97.104172%
Train Epoch: 59/60 Train_Loss: 0.0788467526435852 Train_acc: 97.125000%
Train Epoch: 60/60 Train_Loss: 0.08597011864185333 Train_acc: 97.104172%
Test accuracy of the model: 48.900000%
Test accuracy of the model: 64.416667%
Test accuracy of the model: 71.750000%
Training Time: 87.76 seconds
----------------------------------------------------------------------------------------------------------------
fold:  5 TL:  2
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 1.3853578567504883 Train_acc: 68.475006%
Train Epoch: 2/60 Train_Loss: 0.8774616718292236 Train_acc: 75.012505%
Train Epoch: 3/60 Train_Loss: 0.9324976801872253 Train_acc: 76.187500%
Train Epoch: 4/60 Train_Loss: 0.7908568978309631 Train_acc: 78.912506%
Train Epoch: 5/60 Train_Loss: 0.8787

In [9]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()